In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from transformers import AutoTokenizer
from transformers import AutoModel
from sklearn.metrics.pairwise import cosine_similarity


c:\Users\Arthur\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
links_df = pd.read_csv("C:/Users/Arthur/OneDrive - Jonkoping University/NLP_project/Code/Data/SävsjöAppen linkingtable.csv", delimiter=";")
articles_df = pd.read_csv("C:/Users/Arthur/OneDrive - Jonkoping University/NLP_project/Code/Data/sävsjö_articles.csv")

In [3]:
links_df

,ad_id,tag_id
0,85801,169
1,85801,170
2,85801,171
3,85801,172
4,85801,173
...,...,...
2146,258193,1512
2147,258193,1513
2148,260532,1522
2149,260532,1523


In [4]:
# load numpy array from npy file
from numpy import load
# load array
data = load('sentence_embeddingsv2.npy')
# print the array
print(data)

[[-0.5933311  -0.18265206 -0.18915027 ...  0.23665312  0.78490025
   0.21543747]
 [ 0.21236032  0.31227872  0.22370076 ... -0.25148907  0.30309078
   0.25101122]
 [-0.558921   -0.2906208   0.08355109 ...  0.04057155  0.73291266
  -0.10988649]
 ...
 [-0.31250098 -0.22197773 -0.41341496 ... -0.05273872  0.34888843
  -0.36865178]
 [ 0.10391018 -1.2744927  -0.41148    ... -0.63860166  0.7900625
  -0.82006323]
 [-0.11578982 -1.4671689  -0.71534556 ... -0.63776386  1.0241586
  -0.28924334]]


In [5]:
#retrieve tags that are associated with an article in articles_df
existing_links = links_df[links_df['ad_id'].isin(articles_df['ID'].tolist())]

In [6]:
#unique article ids
len(existing_links['ad_id'].unique())

554

In [7]:
existing_links

,ad_id,tag_id
0,85801,169
1,85801,170
2,85801,171
3,85801,172
4,85801,173
...,...,...
2126,255366,111
2127,255366,573
2128,255392,1160
2129,255392,1159


In [8]:
len(links_df)

2151

In [9]:
#retrieve tags with highest frequency
tags_freq_gb = existing_links.groupby('tag_id').size().reset_index()
tags_freq_gb.rename(columns={0: 'count'}, inplace= True)
tags_freq_df = tags_freq_gb.sort_values('count', ascending=False).reset_index(drop = True)

In [10]:
tags_freq_df

,tag_id,count
0,658,46
1,659,46
2,760,38
3,786,32
4,785,31
...,...,...
799,489,1
800,488,1
801,487,1
802,486,1


In [11]:
sorted_tag_ids = tags_freq_df['tag_id'].tolist()

In [12]:
# sorted_tag_ids

In [13]:
#cosine similarities of all articles
data_cos_sim = cosine_similarity(data)

In [14]:
for tag_id in sorted_tag_ids:
    #retrieving article ids with the current tag
    article_ids = links_df[links_df['tag_id'] == tag_id]['ad_id'].tolist()
    #retrieve index of each article_id from articles_df
    articles_index = articles_df[articles_df['ID'].isin(article_ids)].index.tolist()
    if len(articles_index) <=1:
        continue
    #cross check articles with the same tag and their cosine similarity
    article_cos_sim = data_cos_sim[articles_index[0]] 

    #min_score refers to the lowest cosine similarity score amongst the articles that have the same tag 658
    min_score = 1
    avg_score = 0
    for index in articles_index:
    #     print('Cosine Similarity Score:',article_cos_sim[index])
        if article_cos_sim[index] < min_score:
            min_score = article_cos_sim[index]
        avg_score+= article_cos_sim[index]
    avg_score/= len(articles_index)
    if min_score ==1:
        continue
    #get rank of the lowest score
    article_cos_sim_sorted = sorted(article_cos_sim)

    #higher rank = higher percentile
    rank = article_cos_sim_sorted.index(min_score)
    for i in range(len(article_cos_sim_sorted)-1):
        if avg_score >= article_cos_sim_sorted[i] and avg_score <= article_cos_sim_sorted[i+1]:
            rank_avg = i
            break
    #percentile
    percentile = rank/(len(articles_df))
    percentile_avg = rank_avg/(len(articles_df))
    print("All articles with Tag", tag_id,"are aboove the Percentile:",percentile*100)
    print("Percentile of Averaged Cosine Similarity Scores for articles with Tag", tag_id,":",percentile_avg*100)
    print()

All articles with Tag 658 are aboove the Percentile: 19.349215572341663
Percentile of Averaged Cosine Similarity Scores for articles with Tag 658 : 94.46833236490413

All articles with Tag 659 are aboove the Percentile: 19.349215572341663
Percentile of Averaged Cosine Similarity Scores for articles with Tag 659 : 94.46833236490413

All articles with Tag 760 are aboove the Percentile: 31.5514235909355
Percentile of Averaged Cosine Similarity Scores for articles with Tag 760 : 81.87100522951772

All articles with Tag 786 are aboove the Percentile: 48.64613596746078
Percentile of Averaged Cosine Similarity Scores for articles with Tag 786 : 81.18535735037769

All articles with Tag 785 are aboove the Percentile: 48.64613596746078
Percentile of Averaged Cosine Similarity Scores for articles with Tag 785 : 81.1040092969204

All articles with Tag 829 are aboove the Percentile: 48.64613596746078
Percentile of Averaged Cosine Similarity Scores for articles with Tag 829 : 80.97617664148751

All 

In [15]:
# tag_id = 658
# article_ids = links_df[links_df['tag_id'] == tag_id]['ad_id'].tolist()


In [16]:
# article_ids

In [17]:
#retrieve index of each article_id from articles_df
# articles_index = articles_df[articles_df['ID'].isin(article_ids)].index.tolist()

In [18]:
# articles_index

In [19]:
# #cross check articles with the same tag and their cosine similarity
# #checking tag 658
# article_cos_sim = data_cos_sim[articles_index[0]] 

# min_score = 1
# for index in articles_index:
# #     print('Cosine Similarity Score:',article_cos_sim[index])
#     if article_cos_sim[index] < min_score:
#         min_score = article_cos_sim[index]

In [20]:
#min_score refers to the lowest cosine similarity score amongst the articles that have the same tag 658
# min_score

In [21]:
# #get rank of the lowest score
# article_cos_sim_sorted = sorted(article_cos_sim)

# #higher rank = higher percentile
# rank = article_cos_sim_sorted.index(min_score)
# print(rank)

In [22]:
# article_cos_sim_sorted[2]

In [23]:
# #percentile
# percentile = rank/(len(articles_df))
# print(percentile)

In [24]:
# swedish_stopwords = stopwords.words('swedish')
# #tokenizer for punctuation
# regex_tokenizer = RegexpTokenizer(r'\w+')
# # Load pre-trained model tokenizer (data must be tokenized the same way the model is used)
# tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased")
# # tokenised_data = tokenizer(articles_df['Text'].tolist())

# #loading pretrained model
# model = AutoModel.from_pretrained("KB/bert-base-swedish-cased", output_hidden_states=True)

In [25]:
# tokenised_data.tokens

In [26]:
# tokenised_data[0].tokens

In [27]:
# len(tensors['input_ids'])
# tensors

In [ ]:
# #use only when needed to regenerate sentence embeddings
# import torch
# from tqdm.notebook import tqdm, trange
# # # Put the model in "evaluation" mode, meaning feed-forward operation.
# model.eval()
# sentence_embeddings = []
# MAX_TOKENS = 512

# for i in trange(len(articles_df['Text'])):
#     article_text = articles_df['Text'].iloc[i]
#     word_tokens = regex_tokenizer.tokenize(article_text)
#     filtered_sentence = " ".join([w for w in word_tokens if not w.lower() in swedish_stopwords])
#     filtered_sentence = "[CLS] " + filtered_sentence + " [SEP]"
#     tokenized_article = tokenizer.tokenize(filtered_sentence)
#     indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_article)
#     if len(indexed_tokens) > MAX_TOKENS:
#         #first 511 tokens are retained
#         del indexed_tokens[MAX_TOKENS-1:]
#         #3 is appended as it represents the separation character [SEP] which marks the end of a sentence, 512th token
#         indexed_tokens.append(3)
#     segments_ids = [[1] * len(indexed_tokens)]
#     # Convert inputs to PyTorch tensors
#     tokens_tensor = torch.tensor([indexed_tokens])
#     segments_tensors = torch.tensor(segments_ids)
#     # Predict hidden states features for each layer
#     with torch.no_grad():
#         outputs = model(tokens_tensor, segments_tensors)
#         # Evaluating the model will return a different number of objects based on 
#         # how it's  configured in the `from_pretrained` call earlier. In this case, 
#         # becase we set `output_hidden_states = True`, the third item will be the 
#         # hidden states from all layers. See the documentation for more details:
#         # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
#         hidden_states = outputs[2]
#     # `hidden_states` has shape [13 x 1 x len(segments_id[0]) x 768]
#     # `token_vecs` is a tensor with shape [len(segments_id[0]) x 768]
#     token_vecs = hidden_states[-2][0]

#     # Calculate the average of all 22 token vectors.
#     #convert tensor to np array with .numpy
#     sentence_embedding = torch.mean(token_vecs, dim=0).numpy()
#     sentence_embeddings.append(sentence_embedding)

In [ ]:
# #DO NOT USE THIS
# import torch
# from tqdm.notebook import tqdm, trange
# # # Put the model in "evaluation" mode, meaning feed-forward operation.
# model.eval()
# sentence_embeddings = []
# MAX_TOKENS = 512
# tensors = tokenised_data.convert_to_tensors()

# for i in trange(len(tensors['input_ids'])):
#     indexed_tokens = [tensors['input_ids'][i]]
#     if len(indexed_tokens[0]) > MAX_TOKENS:
#         #first 511 tokens are retained
#         del indexed_tokens[0][MAX_TOKENS-1:]
#         #3 is appended as it represents the separation character [SEP] which marks the end of a sentence, 512th token
#         indexed_tokens[0].append(3)
#     segments_ids = [[1] * len(indexed_tokens[0])]
#     # Convert inputs to PyTorch tensors
#     tokens_tensor = torch.tensor(indexed_tokens)
#     segments_tensors = torch.tensor(segments_ids)
#     # Predict hidden states features for each layer
#     with torch.no_grad():
#         outputs = model(tokens_tensor, segments_tensors)
#         # Evaluating the model will return a different number of objects based on 
#         # how it's  configured in the `from_pretrained` call earlier. In this case, 
#         # becase we set `output_hidden_states = True`, the third item will be the 
#         # hidden states from all layers. See the documentation for more details:
#         # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
#         hidden_states = outputs[2]
#     # `hidden_states` has shape [13 x 1 x 22 x 768]
#     # `token_vecs` is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]

#     # Calculate the average of all 22 token vectors.
#     #convert tensor to np array with .numpy
#     sentence_embedding = torch.mean(token_vecs, dim=0).numpy()
#     sentence_embeddings.append(sentence_embedding)

In [ ]:
# # save numpy array as npy file
# from numpy import asarray
# from numpy import save
# # define data
# data = asarray(sentence_embeddings)
# # save to npy file
# save('sentence_embeddingsv3.npy', data)